In [18]:
"""
FASE 1: Modelo Acoplado Completo + Gravedad
===========================================
Basado en Avilés-Rojas & Hurtado (2025) - JMPS

Ecuaciones:
1. Equilibrio: Div(F·S) + ρ·g = 0                     (Eq. 52 + gravedad)
2. Conservación masa: ∂Φ/∂t + Div(Q) = 0              (Eq. 53)
3. Evolución surfactante: ∂M_surf/∂t = Λ              (Eq. 54)
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
import numpy as np
from firedrake.output import VTKFile
import matplotlib.pyplot as plt

fd.parameters["form_compiler"]["quadrature_degree"] = 6

# ============================================================
# 1. PARÁMETROS DEL MODELO
# ============================================================

# Geometría alveolar
Phi_R = 0.74
R_RI = 0.11

# Material Fung
c_fung = 2.5
a_fung = 0.433
b_fung = -0.61

# Surfactante
gamma_0 = 70.0e-6
gamma_inf = 22.2e-6
gamma_min = 0.0
Gamma_inf = 3.0e-9
m1 = 47.8e-6
m2 = 140.0e-6

# Cinética
k1 = 1.6e8
k2 = 0.016
c_bulk = 1.0e-9

# Darcy
kappa = 1.0e-6
mu_air = 1.8e-8
K_darcy = kappa / mu_air

Gamma_max = Gamma_inf * (1.0 + (gamma_inf - gamma_min) / m2)

# ============================================================
# GRAVEDAD (NUEVO)
# ============================================================

# Densidad del tejido pulmonar
# ρ ≈ 300-400 kg/m³ (pulmón inflado con aire)
# En unidades mm: ρ = 350 kg/m³ = 350e-9 g/mm³ = 3.5e-7 g/mm³
# Para kPa y mm: necesitamos ρ en kg/mm³ → 350e-9 kg/mm³

rho_tissue = 350e-9   # kg/mm³ (densidad del tejido pulmonar)

# Gravedad
g_val = 9810.0        # mm/s² (9.81 m/s² = 9810 mm/s²)

# Vector gravedad (hacia -y en 2D)
# En configuración supina: gravedad en dirección -y
# En configuración prona: gravedad en dirección +y
gravity_direction = fd.Constant((0.0, -1.0))  # Supino (acostado boca arriba)

print("="*60)
print("FASE 1: MODELO ACOPLADO + GRAVEDAD")
print("="*60)
print(f"Φ_R = {Phi_R}, R_I = {R_RI} mm")
print(f"Fung: c = {c_fung} kPa")
print(f"Gravedad:")
print(f"  ρ = {rho_tissue*1e9:.1f} kg/m³")
print(f"  g = {g_val/1000:.2f} m/s²")
print(f"  Dirección: {gravity_direction.values()}")

# Tiempo
T_total = 2.0
dt_val = 0.05
n_steps = int(T_total / dt_val)

print(f"Tiempo: T = {T_total} s, dt = {dt_val} s, pasos = {n_steps}")

# ============================================================
# 2. MALLA Y ESPACIOS
# ============================================================

Lx, Ly = 10.0, 10.0
nx, ny = 20, 20

mesh = fd.RectangleMesh(nx, ny, Lx, Ly)
print(f"\nMalla: {nx}x{ny} elementos, dominio {Lx}x{Ly} mm")

V_u = fd.VectorFunctionSpace(mesh, "CG", 2)
V_P = fd.FunctionSpace(mesh, "CG", 1)
V_M = fd.FunctionSpace(mesh, "DG", 0)

W = V_u * V_P * V_M

w = fd.Function(W)
w_old = fd.Function(W)

u, P_alv, M_surf = fd.split(w)
u_old, P_alv_old, M_surf_old = fd.split(w_old)

v_u, v_P, v_M = fd.TestFunctions(W)

u_out = fd.Function(V_u, name="Displacement")
P_out = fd.Function(V_P, name="Pressure")
M_out = fd.Function(V_M, name="Surfactant_Mass")
Gamma_out = fd.Function(V_M, name="Surfactant_Concentration")
gamma_out = fd.Function(V_M, name="Surface_Tension")

coords_initial = mesh.coordinates.dat.data_ro.copy()

print(f"  DoFs total: {W.dim()}")

# ============================================================
# 3. PARÁMETROS DE TIEMPO
# ============================================================

dt = fd.Constant(dt_val)
t = fd.Constant(0.0)
P_max = fd.Constant(1.0)

# ============================================================
# 4. CINEMÁTICA Y CONSTITUTIVO
# ============================================================

dim = mesh.geometric_dimension()
I = fd.Identity(dim)

F = I + fd.grad(u)
J = fd.det(F)
C = F.T * F
invC = fd.inv(C)

E = fd.variable(0.5 * (C - I))

J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))

Psi_el = c_fung * (fd.exp(a_fung * J1**2 + b_fung * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

Phi = fd.max_value(J - 1.0 + Phi_R, 1e-4)
Phi_old = fd.max_value(fd.det(I + fd.grad(u_old)) - 1.0 + Phi_R, 1e-4)

A_density = (3.0 / R_RI) * (Phi_R**(1.0/3.0)) * (Phi**(2.0/3.0))

Gamma = M_surf / fd.max_value(A_density, 1e-12)

ratio = Gamma / Gamma_inf
gamma_lang = gamma_0 - m1 * ratio
gamma_insol = gamma_inf - m2 * (ratio - 1.0)
gamma_surf = fd.conditional(fd.lt(Gamma, Gamma_inf), gamma_lang, gamma_insol)
gamma_surf = fd.max_value(gamma_surf, gamma_min)

P_gamma = (2.0 * gamma_surf / R_RI) * ((Phi_R / Phi)**(1.0/3.0))

S_total = S_el + (P_gamma - P_alv) * J * invC

# ============================================================
# 5. FLUJO DE DARCY
# ============================================================

Q_darcy = -K_darcy * invC * fd.grad(P_alv)

# ============================================================
# 6. CINÉTICA DEL SURFACTANTE
# ============================================================

Gamma_safe = fd.max_value(Gamma, 0.0)
Lambda_kinetics = k1 * c_bulk * fd.max_value(Gamma_inf - Gamma_safe, 0.0) - k2 * Gamma_safe

# ============================================================
# 7. FUERZA DE GRAVEDAD
# ============================================================

rho_R = fd.Constant(rho_tissue)  # Densidad de referencia
g_vec = g_val * gravity_direction  # Vector gravedad [mm/s²]

# Fuerza de cuerpo en config. referencia (por unidad de volumen ref.)
body_force = rho_R * g_vec

print(f"\nFuerza de gravedad:")
print(f"  |ρ·g| = {float(rho_tissue * g_val):.2e} kPa/mm")

# ============================================================
# 8. FORMULACIÓN VARIACIONAL ACOPLADA
# ============================================================

# Equilibrio mecánico (Eq. 70) + gravedad:
# ∫ F·S : Grad(v) dV - ∫ ρ_R·g · v dV = 0
F_mech = fd.inner(F * S_total, fd.grad(v_u)) * fd.dx \
       - fd.inner(body_force, v_u) * fd.dx  # ← GRAVEDAD

# Conservación de masa (Eq. 71)
F_mass = ((Phi - Phi_old) / dt) * v_P * fd.dx \
       - fd.inner(Q_darcy, fd.grad(v_P)) * fd.dx

# Evolución surfactante (Eq. 72)
F_surf = ((M_surf - M_surf_old) / dt - Lambda_kinetics) * v_M * fd.dx

Residual = F_mech + F_mass + F_surf

# ============================================================
# 9. CONDICIONES DE BORDE
# ============================================================

bcs = [
    fd.DirichletBC(W.sub(0), fd.Constant((0.0, 0.0)), 1),  # x=0 fijo
    fd.DirichletBC(W.sub(0).sub(1), 0.0, 3),               # y=0 simetría
    fd.DirichletBC(W.sub(1), P_max, 2),                    # P en frontera derecha
]

# ============================================================
# 10. SOLVER
# ============================================================

problem = fd.NonlinearVariationalProblem(Residual, w, bcs=bcs)

solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt",
    "snes_max_it": 50,
    "snes_atol": 1e-7,
    "snes_rtol": 1e-5,
    "snes_monitor": None,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}

solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# ============================================================
# 11. CONDICIONES INICIALES
# ============================================================

print("\n" + "="*60)
print("CONDICIONES INICIALES")
print("="*60)

A_density_init = (3.0 / R_RI) * (Phi_R**(1.0/3.0)) * (Phi_R**(2.0/3.0))
M_surf_init = Gamma_max * A_density_init

print(f"  M_surf inicial: {M_surf_init:.6e} g/mm³")
print(f"  Γ inicial: {Gamma_max:.6e} g/mm²")

w.sub(0).assign(fd.Constant((0.0, 0.0)))
w.sub(1).assign(fd.Constant(0.0))
w.sub(2).assign(fd.Constant(M_surf_init))

w_old.assign(w)

# ============================================================
# 12. SIMULACIÓN TRANSITORIA
# ============================================================

print("\n" + "="*60)
print("SIMULACIÓN TRANSITORIA")
print("="*60)

output_dir = "phase1_gravity_results"
os.makedirs(output_dir, exist_ok=True)

outfile = VTKFile(os.path.join(output_dir, "coupled_gravity.pvd"))

times = [0.0]
volumes = [0.0]
pressures_avg = [0.0]
gamma_avg_list = [gamma_inf * 1e6]
M_surf_avg_list = [M_surf_init]
u_y_min_list = [0.0]  # Desplazamiento vertical mínimo (efecto gravedad)

u_sol, P_sol, M_sol = w.subfunctions
u_out.assign(u_sol)
P_out.assign(P_sol)
M_out.assign(M_sol)

Q_out = fd.FunctionSpace(mesh, "DG", 0)
A_init = fd.project(fd.Constant(A_density_init), Q_out)
Gamma_out.assign(fd.project(M_out / A_init, V_M))
gamma_out.assign(fd.project(fd.Constant(gamma_inf), V_M))

outfile.write(u_out, P_out, M_out, Gamma_out, gamma_out)

for step in range(1, n_steps + 1):
    t_val = step * dt_val
    t.assign(t_val)
    
    # Ciclo respiratorio
    if t_val <= T_total / 2:
        P_ext_val = 2.0 * t_val / T_total
    else:
        P_ext_val = 2.0 * (1.0 - t_val / T_total)
    
    P_max.assign(P_ext_val)
    
    print(f"\nPaso {step}/{n_steps}: t = {t_val:.3f} s, P_ext = {P_ext_val:.4f} kPa")
    
    try:
        solver.solve()
    except fd.ConvergenceError:
        print("  ⚠ No convergió")
        break
    
    u_sol, P_sol, M_sol = w.subfunctions
    
    J_curr = fd.project(fd.det(I + fd.grad(u_sol)), Q_out)
    Phi_curr = fd.project(fd.max_value(J_curr - 1.0 + Phi_R, 1e-4), Q_out)
    A_curr = fd.project((3.0 / R_RI) * (Phi_R**(1.0/3.0)) * (Phi_curr**(2.0/3.0)), Q_out)
    
    Gamma_curr = fd.project(M_sol / fd.max_value(A_curr, 1e-12), V_M)
    
    ratio_curr = Gamma_curr / Gamma_inf
    gamma_expr = fd.conditional(fd.lt(Gamma_curr, Gamma_inf),
                                 gamma_0 - m1 * ratio_curr,
                                 gamma_inf - m2 * (ratio_curr - 1.0))
    gamma_curr = fd.project(fd.max_value(gamma_expr, gamma_min), V_M)
    
    u_out.assign(u_sol)
    P_out.assign(P_sol)
    M_out.assign(M_sol)
    Gamma_out.assign(Gamma_curr)
    gamma_out.assign(gamma_curr)
    
    V_change = fd.assemble((J_curr - 1.0) * fd.dx)
    P_avg = fd.assemble(P_sol * fd.dx) / (Lx * Ly)
    gamma_avg = fd.assemble(gamma_curr * fd.dx) / (Lx * Ly)
    M_avg = fd.assemble(M_sol * fd.dx) / (Lx * Ly)
    
    # Desplazamiento vertical (efecto gravedad)
    u_y_data = u_sol.dat.data_ro[:, 1]
    u_y_min = u_y_data.min()
    
    times.append(t_val)
    volumes.append(V_change)
    pressures_avg.append(P_avg)
    gamma_avg_list.append(gamma_avg * 1e6)
    M_surf_avg_list.append(M_avg)
    u_y_min_list.append(u_y_min)
    
    J_min = J_curr.dat.data_ro.min()
    J_max = J_curr.dat.data_ro.max()
    u_max = np.max(np.abs(u_sol.dat.data_ro))
    
    print(f"  |u|_max = {u_max:.4e} mm, u_y_min = {u_y_min:.4e} mm (gravedad)")
    print(f"  J ∈ [{J_min:.4f}, {J_max:.4f}]")
    print(f"  P_avg = {P_avg:.4f} kPa, γ_avg = {gamma_avg*1e6:.2f} μN/mm")
    
    outfile.write(u_out, P_out, M_out, Gamma_out, gamma_out)
    w_old.assign(w)

# ============================================================
# 13. GUARDAR ESTADO DEFORMADO PARA FASE 2
# ============================================================

print("\n" + "="*60)
print("GUARDANDO ESTADO DEFORMADO PARA FASE 2")
print("="*60)

u_final, P_final, M_final = w.subfunctions

# Desplazamiento nodal en la configuración de referencia (malla original)
V_coords = fd.VectorFunctionSpace(mesh, "CG", 1)
u_at_nodes = fd.interpolate(u_final, V_coords)          # u(X) en nodos
u_nodal = u_at_nodes.dat.data_ro.copy()                 # copia segura

# Coordenadas deformadas observadas (configuración tomográfica sintética)
coords_deformed = coords_initial + u_nodal

# Malla deformada sólo para visualización
mesh_deformed = fd.Mesh(mesh.coordinates.copy(deepcopy=True))
mesh_deformed.coordinates.dat.data[:] = coords_deformed

deformed_file = VTKFile(os.path.join(output_dir, "mesh_deformed.pvd"))
V_def = fd.VectorFunctionSpace(mesh_deformed, "CG", 1)
zero_field = fd.Function(V_def, name="zero")
deformed_file.write(zero_field)

# Espacio DG0 para cantidades por celda
Q_out = fd.FunctionSpace(mesh, "DG", 0)

# Campos finales en DG0 (por celda)
J_final     = fd.project(fd.det(I + fd.grad(u_final)), Q_out)
Phi_final   = fd.project(fd.max_value(J_final - 1.0 + Phi_R, 1e-4), Q_out)
A_final     = fd.project((3.0 / R_RI) * (Phi_R**(1.0/3.0)) * (Phi_final**(2.0/3.0)), Q_out)
Gamma_final = fd.project(M_final / fd.max_value(A_final, 1e-12), Q_out)

ratio_final      = Gamma_final / Gamma_inf
gamma_expr_final = fd.conditional(fd.lt(Gamma_final, Gamma_inf),
                                  gamma_0 - m1 * ratio_final,
                                  gamma_inf - m2 * (ratio_final - 1.0))
gamma_final = fd.project(fd.max_value(gamma_expr_final, gamma_min), Q_out)

# Presión alveolar también en DG0 (además de su espacio original CG1)
P_cell_final = fd.project(P_final, Q_out)

# Para la fase 2 es útil tener:
#  - desplazamiento nodal u_nodal (para comparar con w)
#  - coords_initial y coords_deformed (para construir malla observada)
#  - P_alv en CG1 y en DG0
#  - M_surf, Gamma, gamma, J, Phi en DG0

np.savez(os.path.join(output_dir, "forward_data.npz"),
         # Geometría
         coords_initial=coords_initial,         # X (referencia)
         coords_deformed=coords_deformed,       # x_obs
         u_nodal=u_nodal,                       # u(X) en nodos (para u_true)

         # Campos finales (tal como salen de la simulación)
         u_data=u_final.dat.data_ro[:],         # DOFs de u en V_u
         P_alv_data=P_final.dat.data_ro[:],     # DOFs de P en CG1
         M_surf_data=M_final.dat.data_ro[:],    # DOFs de M_surf en DG0

         # Versiones por celda (DG0) para usar en integrales de la fase 2
         P_cell_data=P_cell_final.dat.data_ro[:],   # P_alv proyectado a DG0
         M_cell_data=M_final.dat.data_ro[:],        # M_surf en DG0 (igual que antes)
         Gamma_data=Gamma_final.dat.data_ro[:],     # Γ
         gamma_data=gamma_final.dat.data_ro[:],     # γ
         J_data=J_final.dat.data_ro[:],             # J
         Phi_data=Phi_final.dat.data_ro[:],         # Φ
         A_data=A_final.dat.data_ro[:],             # densidad de área efectiva

         # Parámetros de estado en el tiempo final
         P_alv_final=float(P_max),
         t_final=times[-1],

         # Gravedad
         rho_tissue=rho_tissue,
         g_val=g_val,
         gravity_direction=np.array([0.0, -1.0]),

         # Series temporales (para curvas P–V, etc.)
         times=np.array(times),
         volumes=np.array(volumes),
         pressures=np.array(pressures_avg),
         gamma_avg=np.array(gamma_avg_list),
         M_surf_avg=np.array(M_surf_avg_list),
         u_y_min=np.array(u_y_min_list),

         # Parámetros del modelo (para reconstruir constitutivo en fase 2)
         params=np.array([Phi_R, R_RI, c_fung, a_fung, b_fung,
                          gamma_0, gamma_inf, gamma_min, Gamma_inf, m1, m2,
                          k1, k2, c_bulk, K_darcy]))

print(f"\nEstado final guardado:")
print(f"  t = {times[-1]:.3f} s")
print(f"  P_alv = {float(P_max):.4f} kPa")
print(f"  |u|_max = {np.max(np.abs(u_final.dat.data_ro)):.4f} mm")
print(f"  u_y_min = {u_y_min_list[-1]:.4f} mm (efecto gravedad)")

# ============================================================
# 14. GRÁFICAS
# ============================================================

print("\n" + "="*60)
print("GENERANDO GRÁFICAS")
print("="*60)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(pressures_avg, volumes, 'b-o', markersize=3)
axes[0, 0].set_xlabel('Presión [kPa]')
axes[0, 0].set_ylabel('ΔV [mm²]')
axes[0, 0].set_title('Curva P-V (Histéresis)')
axes[0, 0].grid(True)

axes[0, 1].plot(times, gamma_avg_list, 'g-o', markersize=3)
axes[0, 1].set_xlabel('Tiempo [s]')
axes[0, 1].set_ylabel('γ [μN/mm]')
axes[0, 1].set_title('Tensión Superficial')
axes[0, 1].grid(True)

axes[0, 2].plot(times, u_y_min_list, 'k-o', markersize=3)
axes[0, 2].set_xlabel('Tiempo [s]')
axes[0, 2].set_ylabel('u_y mínimo [mm]')
axes[0, 2].set_title('Efecto de Gravedad (desplazamiento vertical)')
axes[0, 2].grid(True)

axes[1, 0].plot(times, volumes, 'r-o', markersize=3)
axes[1, 0].set_xlabel('Tiempo [s]')
axes[1, 0].set_ylabel('ΔV [mm²]')
axes[1, 0].set_title('Cambio de Volumen')
axes[1, 0].grid(True)

axes[1, 1].plot(times, pressures_avg, 'c-o', markersize=3)
axes[1, 1].set_xlabel('Tiempo [s]')
axes[1, 1].set_ylabel('P_avg [kPa]')
axes[1, 1].set_title('Presión Promedio')
axes[1, 1].grid(True)

axes[1, 2].plot(coords_initial[:, 0], coords_initial[:, 1], 'b.',
                markersize=1, alpha=0.5, label='Inicial')
axes[1, 2].plot(coords_deformed[:, 0], coords_deformed[:, 1], 'r.',
                markersize=1, alpha=0.5, label='Deformado')
axes[1, 2].set_xlabel('x [mm]')
axes[1, 2].set_ylabel('y [mm]')
axes[1, 2].set_title('Config. Inicial vs Deformada (con gravedad)')
axes[1, 2].legend()
axes[1, 2].set_aspect('equal')
axes[1, 2].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "gravity_results.png"), dpi=150)
plt.close()

# ============================================================
# 15. RESUMEN
# ============================================================

print("\n" + "="*60)
print("RESUMEN FINAL")
print("="*60)
print(f"  Pasos completados: {len(times)-1}/{n_steps}")
print(f"  Tiempo final: {times[-1]:.3f} s")
print(f"  ΔV máximo: {max(volumes):.4f} mm²")
print(f"  γ rango: [{min(gamma_avg_list):.2f}, {max(gamma_avg_list):.2f}] μN/mm")

displacement_mag = np.linalg.norm(coords_deformed - coords_initial, axis=1)
print(f"\nDesplazamiento:")
print(f"  Máximo: {displacement_mag.max():.4f} mm")
print(f"  Promedio: {displacement_mag.mean():.4f} mm")

print(f"\nEfecto de gravedad:")
print(f"  u_y mínimo: {min(u_y_min_list):.4f} mm")

print(f"\nArchivos en: {output_dir}/")
print("  - coupled_gravity.pvd: campos temporales")
print("  - mesh_deformed.pvd: malla deformada")
print("  - forward_data.npz: datos para Fase 2 (incluye u_nodal, P_cell, M_cell, etc.)")
print("  - gravity_results.png: gráficas")

print("\n" + "="*60)
print("FASE 1 CON GRAVEDAD COMPLETADA")
print("="*60)
print("  - forward_data.npz: datos para Fase 2 (u, P, M, Γ, γ, etc.)")



FASE 1: MODELO ACOPLADO + GRAVEDAD
Φ_R = 0.74, R_I = 0.11 mm
Fung: c = 2.5 kPa
Gravedad:
  ρ = 350.0 kg/m³
  g = 9.81 m/s²
  Dirección: [ 0. -1.]
Tiempo: T = 2.0 s, dt = 0.05 s, pasos = 40

Malla: 20x20 elementos, dominio 10.0x10.0 mm
  DoFs total: 4603

Fuerza de gravedad:
  |ρ·g| = 3.43e-03 kPa/mm

CONDICIONES INICIALES
  M_surf inicial: 7.014623e-08 g/mm³
  Γ inicial: 3.475714e-09 g/mm²

SIMULACIÓN TRANSITORIA

Paso 1/40: t = 0.050 s, P_ext = 0.0500 kPa
  0 SNES Function norm 1.226665042355e+01
  1 SNES Function norm 4.363675330207e-01
  2 SNES Function norm 2.756293122925e-04
  3 SNES Function norm 3.730235947226e-10
  |u|_max = 8.8895e-02 mm, u_y_min = -3.2366e-02 mm (gravedad)
  J ∈ [0.9984, 1.0327]
  P_avg = 0.0143 kPa, γ_avg = 0.82 μN/mm

Paso 2/40: t = 0.100 s, P_ext = 0.1000 kPa
  0 SNES Function norm 1.227640557290e+01
  1 SNES Function norm 3.666737941661e-01
  2 SNES Function norm 7.767197521688e-05
  |u|_max = 1.5737e-01 mm, u_y_min = -1.8415e-02 mm (gravedad)
  J ∈ [1.00

/home/ebasu/venv-firedrake/lib/python3.12/site-packages/firedrake/interpolation.py:391: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.



Estado final guardado:
  t = 2.000 s
  P_alv = 0.0000 kPa
  |u|_max = 1.0514 mm
  u_y_min = 0.0000 mm (efecto gravedad)

GENERANDO GRÁFICAS

RESUMEN FINAL
  Pasos completados: 40/40
  Tiempo final: 2.000 s
  ΔV máximo: 40.2683 mm²
  γ rango: [0.82, 28.30] μN/mm

Desplazamiento:
  Máximo: 1.1514 mm
  Promedio: 0.6982 mm

Efecto de gravedad:
  u_y mínimo: -0.0324 mm

Archivos en: phase1_gravity_results/
  - coupled_gravity.pvd: campos temporales
  - mesh_deformed.pvd: malla deformada
  - forward_data.npz: datos para Fase 2 (incluye u_nodal, P_cell, M_cell, etc.)
  - gravity_results.png: gráficas

FASE 1 CON GRAVEDAD COMPLETADA
  - forward_data.npz: datos para Fase 2 (u, P, M, Γ, γ, etc.)


In [ ]:
"""
FASE 2: Problema Inverso con parámetros globales θ_P, θ_M
=========================================================

Unknowns:
- w(x): desplazamiento inverso (configuración libre de tensiones)
- θ_P : escala global de presión alveolar
- θ_M : escala global de masa de surfactante

P(x; θ_P)     = θ_P * P_fwd(x)
M_surf(x; θ_M)= θ_M * M_fwd(x)

Cinemática inverse motion en Ω_obs:
- f = I - ∇w
- F = f^{-1}
- J = det(F)
- τ = F S(F, P, M_surf) Fᵀ

Equilibrio en Ω_obs:
∫_Ω_obs τ : ∇v det(f) dx = ∫_Ω_obs ρ_R g · v det(f) dx
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
import numpy as np
from firedrake.output import VTKFile

fd.parameters["form_compiler"]["quadrature_degree"] = 6

# ============================================================
# 1. CARGAR DATOS DE FASE 1
# ============================================================

print("="*60)
print("FASE 2: PROBLEMA INVERSO CON θ_P, θ_M")
print("="*60)

input_dir = "phase1_gravity_results"
data = np.load(os.path.join(input_dir, "forward_data.npz"))

coords_initial_true = data["coords_initial"]   # Ω_R verdadera (rectángulo)
coords_deformed = data["coords_deformed"]      # Ω_obs
P_alv_data = data["P_alv_data"]                # campo P(x) final (CG1)
M_surf_data = data["M_surf_data"]              # campo M_surf(x) final (DG0)
Gamma_data = data["Gamma_data"]                # Γ(x) final (DG0)

params = data["params"]
Phi_R, R_RI = float(params[0]), float(params[1])
c_fung, a_fung, b_fung = float(params[2]), float(params[3]), float(params[4])
gamma_0, gamma_inf, gamma_min = float(params[5]), float(params[6]), float(params[7])
Gamma_inf, m1, m2 = float(params[8]), float(params[9]), float(params[10])

rho_tissue = float(data["rho_tissue"])
g_val = float(data["g_val"])
gravity_direction = data["gravity_direction"]
P_alv_final = float(data["P_alv_final"])

# Desplazamiento "verdadero" usado en el forward sintético
u_true = coords_deformed - coords_initial_true
u_true_mag = np.linalg.norm(u_true, axis=1)

print("\nDatos cargados:")
print(f"  N nodos: {coords_deformed.shape[0]}")
print(f"  P_alv_final = {P_alv_final:.4f} kPa")
print(f"  |u_true|_max  = {u_true_mag.max():.4f} mm")
print(f"  |u_true|_mean = {u_true_mag.mean():.4f} mm")

# ============================================================
# 2. CONSTRUIR MALLA OBSERVADA (Ω_obs) Y REFERENCIA "TRUE"
# ============================================================

# Debe ser compatible con la malla usada en Fase 1: RectangleMesh(nx, ny, 10, 10)
Lx, Ly = 10.0, 10.0
nx, ny = 20, 20

mesh_base = fd.RectangleMesh(nx, ny, Lx, Ly)  # Ω_R rectangular "true"
mesh_obs = fd.Mesh(mesh_base.coordinates.copy(deepcopy=True))
mesh_obs.coordinates.dat.data[:] = coords_deformed  # Ω_obs (tomografía)

V_obs = fd.assemble(fd.Constant(1.0) * fd.dx(domain=mesh_obs))
V_true_sf = fd.assemble(fd.Constant(1.0) * fd.dx(domain=mesh_base))

print("\nVolúmenes:")
print(f"  V_obs (deformado)  = {V_obs:.4f} mm²")
print(f"  V_sf (verdadero)   = {V_true_sf:.4f} mm²")
print(f"  V_obs/V_sf (esperado) = {V_obs/V_true_sf:.3f}")

# ============================================================
# 3. ESPACIOS DE FUNCIONES
# ============================================================

# w(x) en CG2 sobre Ω_obs
V_w = fd.VectorFunctionSpace(mesh_obs, "CG", 2)

# Parámetros globales θ_P, θ_M en espacio "R" (1 dof global)
R_space = fd.FunctionSpace(mesh_obs, "R", 0)

# Campos base P_fwd (CG1), M_fwd (DG0) sobre Ω_obs
V_P = fd.FunctionSpace(mesh_obs, "CG", 1)
V_M = fd.FunctionSpace(mesh_obs, "DG", 0)

P_fwd = fd.Function(V_P, name="P_forward")
M_fwd = fd.Function(V_M, name="M_forward")
Gamma_fwd = fd.Function(V_M, name="Gamma_forward")

# Asignar datos desde Fase 1
P_fwd.dat.data[:] = P_alv_data
M_fwd.dat.data[:] = M_surf_data
Gamma_fwd.dat.data[:] = Gamma_data

print("\nCampos base (Fase 1):")
print(f"  ||P_fwd||_max = {np.abs(P_fwd.dat.data_ro).max():.4e} kPa")
print(f"  ||M_fwd||_max = {np.abs(M_fwd.dat.data_ro).max():.4e} g/mm²")

# Espacio mixto para (w, θ_P, θ_M)
W = V_w * R_space * R_space
state = fd.Function(W, name="Inverse_State")

w, theta_P, theta_M = fd.split(state)
v_w, q_P, q_M = fd.TestFunctions(W)

dim = mesh_obs.geometric_dimension()
I = fd.Identity(dim)

# Subfunciones para inicializar y postprocesar
w_func, thetaP_func, thetaM_func = state.subfunctions

# ============================================================
# 4. CONSTANTES Y PARÁMETROS
# ============================================================

rho_R = fd.Constant(rho_tissue)  # densidad de referencia
g_vec = fd.Constant(tuple(g_val * gravity_direction))  # vector gravedad

Gamma_inf_c = fd.Constant(Gamma_inf)
gamma_0_c = fd.Constant(gamma_0)
gamma_inf_c = fd.Constant(gamma_inf)
gamma_min_c = fd.Constant(gamma_min)
m1_c = fd.Constant(m1)
m2_c = fd.Constant(m2)
Phi_R_c = fd.Constant(Phi_R)
R_RI_c = fd.Constant(R_RI)
c_fung_c = fd.Constant(c_fung)
a_fung_c = fd.Constant(a_fung)
b_fung_c = fd.Constant(b_fung)

# Regularización
alpha_reg_w = fd.Constant(1e-2)    # suavidad w
lambda_P_reg = fd.Constant(1e-2)   # Tikhonov para θ_P
lambda_M_reg = fd.Constant(1e-2)   # Tikhonov para θ_M

# Parámetro de continuación en las cargas
alpha_load = fd.Constant(0.0)

print("\nGravedad:")
print(f"  ρ = {rho_tissue*1e9:.1f} kg/m³")
print(f"  g = {g_val/1000:.2f} m/s²")
print(f"  Dirección: {gravity_direction}")

# ============================================================
# 5. CINEMÁTICA INVERSE MOTION
# ============================================================

# f = I - ∇w  (gradiente del mapeo inverso X = x - w(x))
grad_w = fd.grad(w)
f = I - grad_w
det_f = fd.det(f)

# F = ∂x/∂X = f^{-1}
F = fd.inv(f)
J = fd.det(F)
C = F.T * F
invC = fd.inv(C)

# Tensor de Green-Lagrange
E = fd.variable(0.5 * (C - I))
J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))

# Energía Fung y segundo tensor de Piola elástico
Psi_el = c_fung_c * (fd.exp(a_fung_c * J1**2 + b_fung_c * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

# Porosidad en configuración de referencia (depende de J)
Phi = fd.max_value(J - 1.0 + Phi_R_c, 1e-4)

# Área interfacial por volumen (como en Fase 1, pero en Ω_R)
A_density = (3.0 / R_RI_c) * (Phi_R_c**(1.0/3.0)) * (Phi**(2.0/3.0))

# ============================================================
# 6. CAMPOS P(x; θ_P) Y M_surf(x; θ_M), SURFACTANTE Y P_γ
# ============================================================

# θ_P y θ_M son globales (espacio R), pero se evalúan como escalares
P_alv_eff = theta_P * P_fwd        # P(x; θ_P)
M_surf_eff = theta_M * M_fwd       # M_surf(x; θ_M)

Gamma = M_surf_eff / fd.max_value(A_density, 1e-12)

ratio = Gamma / Gamma_inf_c
gamma_lang = gamma_0_c - m1_c * ratio
gamma_insol = gamma_inf_c - m2_c * (ratio - 1.0)
gamma_surf = fd.conditional(fd.lt(Gamma, Gamma_inf_c), gamma_lang, gamma_insol)
gamma_surf = fd.max_value(gamma_surf, gamma_min_c)

P_gamma = (2.0 * gamma_surf / R_RI_c) * ((Phi_R_c / Phi)**(1.0/3.0))

# Segundo Piola total con cargas escaladas por alpha_load
S_total = S_el + alpha_load * (P_gamma - P_alv_eff) * J * invC

# Tensor de Kirchhoff τ = F S Fᵀ
tau = F * S_total * F.T

# Fuerza de cuerpo escalada
body_force = alpha_load * rho_R * g_vec

# ============================================================
# 7. FORMULACIÓN VARIACIONAL (Ω_obs)
# ============================================================

# Residuo mecánico en Ω_obs:
# R_w = ∫ τ : ∇v_w det(f) dx - ∫ (ρ_R g) · v_w det(f) dx
Residual = fd.inner(tau, fd.grad(v_w)) * det_f * fd.dx \
         - fd.inner(body_force, v_w) * det_f * fd.dx

# Penalización de borde (similar a Fase 1: anclar izquierda y fondo)
x = fd.SpatialCoordinate(mesh_obs)
x_min = coords_deformed[:, 0].min()
x_max = coords_deformed[:, 0].max()
y_min = coords_deformed[:, 1].min()

tol = 0.05 * (x_max - x_min)

indicator_left = fd.conditional(fd.lt(x[0], x_min + tol), 1.0, 0.0)
indicator_bottom = fd.conditional(fd.lt(x[1], y_min + tol), 1.0, 0.0)

beta_bc = fd.Constant(1e6)

Residual += beta_bc * indicator_left * fd.inner(w, v_w) * fd.dx
Residual += beta_bc * indicator_bottom * w[1] * v_w[1] * fd.dx

# Regularización en w (suavidad)
Residual += alpha_reg_w * fd.inner(fd.grad(w), fd.grad(v_w)) * fd.dx

# Ecuaciones "suaves" para θ_P y θ_M: Tikhonov hacia 1.0
# (Esto mantiene θ ≈ 1 mientras no incluyamos un functional de mismatch geométrico)
Residual += lambda_P_reg * (theta_P - 1.0) * q_P * fd.dx
Residual += lambda_M_reg * (theta_M - 1.0) * q_M * fd.dx

# ============================================================
# 8. SOLVER NO LINEAL
# ============================================================

problem = fd.NonlinearVariationalProblem(Residual, state)

solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt",
    "snes_max_it": 50,
    "snes_atol": 1e-6,
    "snes_rtol": 1e-5,
    "snes_monitor": None,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}

solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# Inicialización
w_func.assign(fd.Constant((0.0, 0.0)))
thetaP_func.assign(1.0)
thetaM_func.assign(1.0)

# ============================================================
# 9. CONTINUACIÓN EN LAS CARGAS (α = 0 → 1)
# ============================================================

print("\n" + "="*60)
print("RESOLUCIÓN PROBLEMA INVERSO (CONTINUACIÓN EN α)")
print("="*60)

n_steps = 10

for step in range(1, n_steps + 1):
    alpha_val = step / n_steps
    alpha_load.assign(alpha_val)

    print(f"\nPaso {step}/{n_steps}: α = {alpha_val:.2f}")
    try:
        solver.solve()
        w_data = w_func.dat.data_ro
        w_mag = np.linalg.norm(w_data, axis=1)
        thetaP_val = thetaP_func.dat.data_ro[0]
        thetaM_val = thetaM_func.dat.data_ro[0]
        print(f"  ✓ |w|_max = {w_mag.max():.4e} mm, |w|_mean = {w_mag.mean():.4e} mm")
        print(f"  θ_P = {thetaP_val:.4f}, θ_M = {thetaM_val:.4f}")
    except fd.ConvergenceError:
        print("  ✗ No convergió")
        break

# ============================================================
# 10. CONFIGURACIÓN STRESS-FREE RECUPERADA
# ============================================================

print("\n" + "="*60)
print("CONFIGURACIÓN STRESS-FREE RECUPERADA")
print("="*60)

V_coords = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
w_interp = fd.interpolate(w_func, V_coords)
w_at_nodes = w_interp.dat.data_ro

coords_sf = coords_deformed - w_at_nodes  # X = x - w(x)

# Malla stress-free recuperada
mesh_sf = fd.Mesh(mesh_obs.coordinates.copy(deepcopy=True))
mesh_sf.coordinates.dat.data[:] = coords_sf

V_sf = fd.assemble(fd.Constant(1.0) * fd.dx(domain=mesh_sf))

print("\nVolúmenes:")
print(f"  V_obs         = {V_obs:.4f} mm²")
print(f"  V_sf recup.   = {V_sf:.4f} mm²")
print(f"  V_sf verdadero= {V_true_sf:.4f} mm²")
print(f"  V_obs/V_sf recup. = {V_obs/V_sf:.3f}")
print(f"  V_obs/V_sf true   = {V_obs/V_true_sf:.3f}")

if V_sf < V_obs:
    print("  ✓ V_sf < V_obs (consistente con configuración libre de tensiones)")
else:
    print("  ⚠ V_sf >= V_obs (revisar cargas/regularización si quieres más contracción)")

# Error geométrico vs configuración verdadera
error_sf = coords_sf - coords_initial_true
error_sf_mag = np.linalg.norm(error_sf, axis=1)

x_max = coords_deformed[:, 0].max()
y_max = coords_deformed[:, 1].max()
L_char = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)

print("\nError de recuperación (coords):")
print(f"  Error máximo   = {error_sf_mag.max():.6f} mm ({error_sf_mag.max()/L_char*100:.2f} %)")
print(f"  Error promedio = {error_sf_mag.mean():.6f} mm")

w_mag = np.linalg.norm(w_at_nodes, axis=1)
print("\nDesplazamientos:")
print(f"  |u_true|_max = {u_true_mag.max():.4f} mm")
print(f"  |w|_max      = {w_mag.max():.4f} mm")
print(f"  |w|_mean     = {w_mag.mean():.4f} mm")

thetaP_val = thetaP_func.dat.data_ro[0]
thetaM_val = thetaM_func.dat.data_ro[0]
print("\nParámetros globales:")
print(f"  θ_P = {thetaP_val:.4f}")
print(f"  θ_M = {thetaM_val:.4f}")

# ============================================================
# 11. GUARDAR RESULTADOS
# ============================================================

output_dir = "phase2_inverse_results"
os.makedirs(output_dir, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO RESULTADOS")
print("="*60)

# Campos P_eff y M_eff resultantes
P_eff = fd.Function(V_P, name="P_eff")
M_eff = fd.Function(V_M, name="M_eff")
P_eff.assign(thetaP_val * P_fwd)
M_eff.assign(thetaM_val * M_fwd)

# Archivos VTK
obs_file = VTKFile(os.path.join(output_dir, "mesh_observed.pvd"))
sf_file = VTKFile(os.path.join(output_dir, "mesh_stressfree.pvd"))

V_obs_out = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
recovery_vec = fd.Function(V_obs_out, name="Recovery_Vector")
recovery_vec.dat.data[:] = -w_at_nodes   # X = x - w ⇒ vector desde obs a SF = -w
obs_file.write(recovery_vec, P_eff, M_eff)

V_sf_out = fd.VectorFunctionSpace(mesh_sf, "CG", 1)
zero_sf = fd.Function(V_sf_out, name="zero")
sf_file.write(zero_sf)

# Datos en NPZ
np.savez(os.path.join(output_dir, "inverse_data.npz"),
         coords_observed=coords_deformed,
         coords_stressfree=coords_sf,
         coords_stressfree_true=coords_initial_true,
         w_data=w_at_nodes,
         thetaP=thetaP_val,
         thetaM=thetaM_val,
         P_fwd=P_alv_data,
         M_fwd=M_surf_data,
         P_eff=P_eff.dat.data_ro[:],
         M_eff=M_eff.dat.data_ro[:],
         error_sf=error_sf_mag)

print("\n" + "="*60)
print("RESUMEN FINAL FASE 2 INVERSA")
print("="*60)
print(f"\nVolúmenes:")
print(f"  V_obs       = {V_obs:.4f} mm²")
print(f"  V_sf recup. = {V_sf:.4f} mm²")
print(f"  V_sf true   = {V_true_sf:.4f} mm²")

print(f"\nDesplazamientos:")
print(f"  |u_true|_max = {u_true_mag.max():.4f} mm")
print(f"  |w|_max      = {w_mag.max():.4f} mm")
print(f"  |w|_mean     = {w_mag.mean():.4f} mm")

print(f"\nParámetros:")
print(f"  θ_P = {thetaP_val:.4f}")
print(f"  θ_M = {thetaM_val:.4f}")

print(f"\nErrores geométricos:")
print(f"  Error máx    = {error_sf_mag.max():.4f} mm ({error_sf_mag.max()/L_char*100:.2f} %)")
print(f"  Error medio  = {error_sf_mag.mean():.4f} mm")

print(f"\nArchivos en: {output_dir}/")
print("  - mesh_observed.pvd     (Recovery_Vector, P_eff, M_eff)")
print("  - mesh_stressfree.pvd   (configuración SF recuperada)")
print("  - inverse_data.npz      (datos numéricos)")
print("\nParaView:")
print("  1. Abrir mesh_observed.pvd")
print("  2. Warp By Vector → Recovery_Vector (scale ≈ 1)")
print("  3. Debería contraer hacia la configuración libre de tensiones")
print("\n" + "="*60)
print("FASE 2 INVERSA CON PARÁMETROS θ_P, θ_M COMPLETADA")
print("="*60)


FASE 2: PROBLEMA INVERSO - FORMULACIÓN CORRECTA

Datos cargados:
  Nodos: 441
  P_alv = 0.0000 kPa
  |u_true|_max  = 1.1514 mm
  |u_true|_mean = 0.6982 mm

Volúmenes:
  V_obs (deformado) = 116.6476 mm²
  V_sf (verdadero)  = 100.0000 mm²
  Ratio esperado V_obs/V_sf = 1.166

RESOLVIENDO PROBLEMA INVERSO

Paso 1/10: α = 0.10
  0 SNES Function norm 1.156124608259e-03
  1 SNES Function norm 2.770930032175e-05
  2 SNES Function norm 4.068066269581e-10
  ✓ |w|_max  = 0.0088 mm
    |w|_mean = 0.0036 mm
    Correlación dirección w·u_true ≈ -0.2317 (ideal ≈ 1)

Paso 2/10: α = 0.20
  0 SNES Function norm 1.156575124316e-03
  1 SNES Function norm 2.789558523282e-05
  2 SNES Function norm 4.152627522292e-10
  ✓ |w|_max  = 0.0178 mm
    |w|_mean = 0.0073 mm
    Correlación dirección w·u_true ≈ -0.2321 (ideal ≈ 1)

Paso 3/10: α = 0.30
  0 SNES Function norm 1.157028687666e-03
  1 SNES Function norm 2.808473701587e-05


/home/ebasu/venv-firedrake/lib/python3.12/site-packages/firedrake/interpolation.py:391: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.


  2 SNES Function norm 4.239721368777e-10
  ✓ |w|_max  = 0.0267 mm
    |w|_mean = 0.0109 mm
    Correlación dirección w·u_true ≈ -0.2324 (ideal ≈ 1)

Paso 4/10: α = 0.40
  0 SNES Function norm 1.157485316862e-03
  1 SNES Function norm 2.827682444746e-05
  2 SNES Function norm 4.329457354454e-10
  ✓ |w|_max  = 0.0357 mm
    |w|_mean = 0.0146 mm
    Correlación dirección w·u_true ≈ -0.2328 (ideal ≈ 1)

Paso 5/10: α = 0.50
  0 SNES Function norm 1.157945048164e-03
  1 SNES Function norm 2.847191536424e-05
  2 SNES Function norm 4.421932672179e-10
  ✓ |w|_max  = 0.0448 mm
    |w|_mean = 0.0183 mm
    Correlación dirección w·u_true ≈ -0.2332 (ideal ≈ 1)

Paso 6/10: α = 0.60
  0 SNES Function norm 1.158407918499e-03
  1 SNES Function norm 2.867007969169e-05
  2 SNES Function norm 4.517259243778e-10
  ✓ |w|_max  = 0.0540 mm
    |w|_mean = 0.0220 mm
    Correlación dirección w·u_true ≈ -0.2335 (ideal ≈ 1)

Paso 7/10: α = 0.70
  0 SNES Function norm 1.158873965487e-03
  1 SNES Function norm 2.8

In [35]:
"""
FASE 2: Problema Inverso con parámetros globales θ_P, θ_M + Volumen objetivo
============================================================================

Unknowns:
- w(x): desplazamiento inverso (configuración libre de tensiones)
- θ_P : escala global de presión alveolar
- θ_M : escala global de masa de surfactante
- λ_vol : multiplicador de Lagrange para volumen stress-free

P(x; θ_P)      = θ_P * P_fwd(x)
M_surf(x; θ_M) = θ_M * M_fwd(x)

Cinemática inverse motion en Ω_obs:
- f = I - ∇w
- F = f^{-1}
- J = det(F)
- τ = F S(F, P, M_surf) Fᵀ

Equilibrio en Ω_obs:
∫_Ω_obs τ : ∇v det(f) dx = ∫_Ω_obs ρ_R g · v det(f) dx

Volumen objetivo (en Ω_R):
∫_Ω_obs det(f) dx = V_sf,true
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
import numpy as np
from firedrake.output import VTKFile

fd.parameters["form_compiler"]["quadrature_degree"] = 6

# ============================================================
# 1. CARGAR DATOS DE FASE 1
# ============================================================

print("="*60)
print("FASE 2: PROBLEMA INVERSO CON θ_P, θ_M + VOLUMEN OBJETIVO")
print("="*60)

input_dir = "phase1_gravity_results"
data = np.load(os.path.join(input_dir, "forward_data.npz"))

coords_initial_true = data["coords_initial"]   # Ω_R verdadera (rectángulo)
coords_deformed = data["coords_deformed"]      # Ω_obs
P_alv_data = data["P_alv_data"]                # campo P(x) final (CG1)
M_surf_data = data["M_surf_data"]              # campo M_surf(x) final (DG0)
Gamma_data = data["Gamma_data"]                # Γ(x) final (DG0)

params = data["params"]
Phi_R, R_RI = float(params[0]), float(params[1])
c_fung, a_fung, b_fung = float(params[2]), float(params[3]), float(params[4])
gamma_0, gamma_inf, gamma_min = float(params[5]), float(params[6]), float(params[7])
Gamma_inf, m1, m2 = float(params[8]), float(params[9]), float(params[10])

rho_tissue = float(data["rho_tissue"])
g_val = float(data["g_val"])
gravity_direction = data["gravity_direction"]
P_alv_final = float(data["P_alv_final"])

# Desplazamiento "verdadero" usado en el forward sintético
u_true = coords_deformed - coords_initial_true
u_true_mag = np.linalg.norm(u_true, axis=1)

print("\nDatos cargados:")
print(f"  N nodos: {coords_deformed.shape[0]}")
print(f"  P_alv_final = {P_alv_final:.4f} kPa")
print(f"  |u_true|_max  = {u_true_mag.max():.4f} mm")
print(f"  |u_true|_mean = {u_true_mag.mean():.4f} mm")

# ============================================================
# 2. CONSTRUIR MALLAS Ω_R (true) Y Ω_obs
# ============================================================

Lx, Ly = 10.0, 10.0
nx, ny = 20, 20

mesh_base = fd.RectangleMesh(nx, ny, Lx, Ly)  # Ω_R rectangular "true"
mesh_obs = fd.Mesh(mesh_base.coordinates.copy(deepcopy=True))
mesh_obs.coordinates.dat.data[:] = coords_deformed  # Ω_obs (tomografía)

V_obs_form = fd.Constant(1.0) * fd.dx(domain=mesh_obs)
V_true_form = fd.Constant(1.0) * fd.dx(domain=mesh_base)

V_obs = fd.assemble(V_obs_form)
V_true_sf = fd.assemble(V_true_form)

V_obs_val = float(V_obs)
V_true_sf_val = float(V_true_sf)

print("\nVolúmenes:")
print(f"  V_obs (deformado)  = {V_obs_val:.4f} mm²")
print(f"  V_sf (verdadero)   = {V_true_sf_val:.4f} mm²")
print(f"  V_obs/V_sf (esperado) = {V_obs_val/V_true_sf_val:.3f}")

# Target global de volumen para la configuración stress-free
# (En pulmón: aquí pondrías el volumen SF esperado a partir de la curva P–V)
V_sf_target = V_true_sf_val

# Densidad objetivo de jacobiano: det(f) debería promediarse a V_sf_target / V_obs
V_target_density = fd.Constant(V_sf_target / V_obs_val)

# Centro de masa verdadero (para penalización COM opcional)
com_true = coords_initial_true.mean(axis=0)
com_true_vec = fd.Constant((float(com_true[0]), float(com_true[1])))

# ============================================================
# 3. ESPACIOS DE FUNCIONES
# ============================================================

# w(x) en CG2 sobre Ω_obs
V_w = fd.VectorFunctionSpace(mesh_obs, "CG", 2)

# Parámetros globales en espacio R (1 dof global)
R_space = fd.FunctionSpace(mesh_obs, "R", 0)

# Campos base P_fwd (CG1), M_fwd (DG0) sobre Ω_obs
V_P = fd.FunctionSpace(mesh_obs, "CG", 1)
V_M = fd.FunctionSpace(mesh_obs, "DG", 0)

P_fwd = fd.Function(V_P, name="P_forward")
M_fwd = fd.Function(V_M, name="M_forward")
Gamma_fwd = fd.Function(V_M, name="Gamma_forward")

# Asignar datos desde Fase 1
P_fwd.dat.data[:] = P_alv_data
M_fwd.dat.data[:] = M_surf_data
Gamma_fwd.dat.data[:] = Gamma_data

print("\nCampos base (Fase 1):")
print(f"  ||P_fwd||_max = {np.abs(P_fwd.dat.data_ro).max():.4e} kPa")
print(f"  ||M_fwd||_max = {np.abs(M_fwd.dat.data_ro).max():.4e} g/mm²")

# Espacio mixto para (w, θ_P, θ_M, λ_vol)
W = V_w * R_space * R_space * R_space
state = fd.Function(W, name="Inverse_State")

w, theta_P, theta_M, lambda_vol = fd.split(state)
v_w, q_P, q_M, q_lambda = fd.TestFunctions(W)

# Subfunciones para inicializar y postprocesar
w_func, thetaP_func, thetaM_func, lambda_vol_func = state.subfunctions

dim = mesh_obs.geometric_dimension()
I = fd.Identity(dim)

# ============================================================
# 4. CONSTANTES Y PARÁMETROS
# ============================================================

rho_R = fd.Constant(rho_tissue)  # densidad de referencia
g_vec = fd.Constant(tuple(g_val * gravity_direction))  # vector gravedad

Gamma_inf_c = fd.Constant(Gamma_inf)
gamma_0_c = fd.Constant(gamma_0)
gamma_inf_c = fd.Constant(gamma_inf)
gamma_min_c = fd.Constant(gamma_min)
m1_c = fd.Constant(m1)
m2_c = fd.Constant(m2)
Phi_R_c = fd.Constant(Phi_R)
R_RI_c = fd.Constant(R_RI)
c_fung_c = fd.Constant(c_fung)
a_fung_c = fd.Constant(a_fung)
b_fung_c = fd.Constant(b_fung)

# Regularización
alpha_reg_w = fd.Constant(1e-3)   # suavidad w (strain-like)
lambda_P_reg = fd.Constant(1e-2)  # Tikhonov para θ_P
lambda_M_reg = fd.Constant(1e-2)  # Tikhonov para θ_M

# Penalización centro de masa (opcional)
gamma_com = fd.Constant(0.0)      # si quieres activarla: 1e-3, por ejemplo

# Parámetro de continuación en las cargas
alpha_load = fd.Constant(0.0)

print("\nGravedad:")
print(f"  ρ = {rho_tissue*1e9:.1f} kg/m³")
print(f"  g = {g_val/1000:.2f} m/s²")
print(f"  Dirección: {gravity_direction}")

# ============================================================
# 5. CINEMÁTICA INVERSE MOTION
# ============================================================

# f = I - ∇w  (gradiente del mapeo inverso X = x - w(x))
grad_w = fd.grad(w)
f = I - grad_w
det_f = fd.det(f)

# F = ∂x/∂X = f^{-1}
F = fd.inv(f)
J = fd.det(F)
C = F.T * F
invC = fd.inv(C)

# Tensor de Green-Lagrange
E = fd.variable(0.5 * (C - I))
J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))

# Energía Fung y segundo tensor de Piola elástico
Psi_el = c_fung_c * (fd.exp(a_fung_c * J1**2 + b_fung_c * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

# Porosidad en configuración de referencia (depende de J)
Phi = fd.max_value(J - 1.0 + Phi_R_c, 1e-4)

# Área interfacial por volumen (como en Fase 1)
A_density = (3.0 / R_RI_c) * (Phi_R_c**(1.0/3.0)) * (Phi**(2.0/3.0))

# ============================================================
# 6. CAMPOS P(x; θ_P), M_surf(x; θ_M), SURFACTANTE Y P_γ
# ============================================================

P_alv_eff = theta_P * P_fwd        # P(x; θ_P)
M_surf_eff = theta_M * M_fwd       # M_surf(x; θ_M)

Gamma = M_surf_eff / fd.max_value(A_density, 1e-12)

ratio = Gamma / Gamma_inf_c
gamma_lang = gamma_0_c - m1_c * ratio
gamma_insol = gamma_inf_c - m2_c * (ratio - 1.0)
gamma_surf = fd.conditional(fd.lt(Gamma, Gamma_inf_c), gamma_lang, gamma_insol)
gamma_surf = fd.max_value(gamma_surf, gamma_min_c)

P_gamma = (2.0 * gamma_surf / R_RI_c) * ((Phi_R_c / Phi)**(1.0/3.0))

# Segundo Piola total con cargas escaladas por alpha_load
S_total = S_el + alpha_load * (P_gamma - P_alv_eff) * J * invC

# Tensor de Kirchhoff τ = F S Fᵀ
tau = F * S_total * F.T

# Fuerza de cuerpo escalada
body_force = alpha_load * rho_R * g_vec





# ============================================================
# 7. FORMULACIÓN VARIACIONAL (Ω_obs)
# ============================================================

x = fd.SpatialCoordinate(mesh_obs)

# Residuo mecánico en Ω_obs:
Residual = fd.inner(tau, fd.grad(v_w)) * det_f * fd.dx \
         - fd.inner(body_force, v_w) * det_f * fd.dx
         
         

# Penalización de borde (anclar izquierda y fondo)
x_min = coords_deformed[:, 0].min()
x_max = coords_deformed[:, 0].max()
y_min = coords_deformed[:, 1].min()

tol = 0.05 * (x_max - x_min)

indicator_left = fd.conditional(fd.lt(x[0], x_min + tol), 1.0, 0.0)
indicator_bottom = fd.conditional(fd.lt(x[1], y_min + tol), 1.0, 0.0)

beta_bc = fd.Constant(1e7)

Residual += beta_bc * indicator_left * fd.inner(w, v_w) * fd.dx
Residual += beta_bc * indicator_bottom * w[1] * v_w[1] * fd.dx

# Regularización strain-like en w
eps_w = 0.5 * (grad_w + grad_w.T)
eps_v = 0.5 * (fd.grad(v_w) + fd.grad(v_w).T)
Residual += alpha_reg_w * fd.inner(eps_w, eps_v) * fd.dx

# Tikhonov para θ_P y θ_M
Residual += lambda_P_reg * (theta_P - 1.0) * q_P * fd.dx
Residual += lambda_M_reg * (theta_M - 1.0) * q_M * fd.dx

# ============================================================
# 8. RESTRICCIÓN DE VOLUMEN GLOBAL (LAGRANGE λ_vol)
# ============================================================

# Form de volumen en Ω_obs: ∫ det(f) dx
vol_form_det = det_f * fd.dx

# Contribución al equilibrio de w: λ_vol * d/dw(∫ det(f) dx)
# Avoid multiplying an Indexed (lambda_vol) by an already-built Form (vol_form_det).
# Instead build the form using the UFL expression det_f and fd.dx directly.
Residual += fd.derivative(lambda_vol * det_f * fd.dx, w, v_w)

# Ecuación para λ_vol: ∫ (det(f) - V_target_density) dx = 0
# Esto impone ∫ det(f) dx = V_sf_target
Residual += (det_f - V_target_density) * q_lambda * fd.dx

# ============================================================
# 9. PENALIZACIÓN CENTRO DE MASA (OPCIONAL)
# ============================================================

# X_sf(x) = x - w(x) (aprox. coordenadas en configuración SF)
X_sf = x - w
Residual += gamma_com * fd.inner(X_sf - com_true_vec, v_w) * det_f * fd.dx

# ============================================================
# 10. SOLVER NO LINEAL (NEST + FIELDSPLIT)
# ============================================================

problem = fd.NonlinearVariationalProblem(Residual, state)

solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt",
    "snes_max_it": 50,
    "snes_atol": 1e-6,
    "snes_rtol": 1e-5,
    "snes_monitor": None,

    "mat_type": "nest",
    "ksp_type": "fgmres",
    "ksp_max_it": 20,

    "pc_type": "fieldsplit",
    "pc_fieldsplit_type": "multiplicative",

    # Bloque 0: w
    "fieldsplit_0_ksp_type": "preonly",
    "fieldsplit_0_pc_type": "lu",
    "fieldsplit_0_pc_factor_mat_solver_type": "mumps",

    # Bloque 1: θ_P
    "fieldsplit_1_ksp_type": "preonly",
    "fieldsplit_1_pc_type": "jacobi",

    # Bloque 2: θ_M
    "fieldsplit_2_ksp_type": "preonly",
    "fieldsplit_2_pc_type": "jacobi",

    # Bloque 3: λ_vol
    "fieldsplit_3_ksp_type": "preonly",
    "fieldsplit_3_pc_type": "jacobi",
}

solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# Inicialización
w_func.assign(fd.Constant((0.0, 0.0)))
thetaP_func.assign(1.0)
thetaM_func.assign(1.0)
lambda_vol_func.assign(0.0)

# ============================================================
# 11. CONTINUACIÓN EN LAS CARGAS (α = 0 → 1)
# ============================================================

print("\n" + "="*60)
print("RESOLUCIÓN PROBLEMA INVERSO (CONTINUACIÓN EN α)")
print("="*60)

n_steps = 10

for step in range(1, n_steps + 1):
    alpha_val = step / n_steps
    alpha_load.assign(alpha_val)

    print(f"\nPaso {step}/{n_steps}: α = {alpha_val:.2f}")
    try:
        solver.solve()
        w_data = w_func.dat.data_ro
        w_mag = np.linalg.norm(w_data, axis=1)
        thetaP_val = thetaP_func.dat.data_ro[0]
        thetaM_val = thetaM_func.dat.data_ro[0]
        lambda_vol_val = lambda_vol_func.dat.data_ro[0]
        print(f"  ✓ |w|_max = {w_mag.max():.4e} mm, |w|_mean = {w_mag.mean():.4e} mm")
        print(f"    θ_P = {thetaP_val:.4f}, θ_M = {thetaM_val:.4f}, λ_vol = {lambda_vol_val:.4e}")
    except fd.ConvergenceError:
        print("  ✗ No convergió")
        break

# ============================================================
# 12. CONFIGURACIÓN STRESS-FREE RECUPERADA
# ============================================================

print("\n" + "="*60)
print("CONFIGURACIÓN STRESS-FREE RECUPERADA")
print("="*60)

V_coords = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
w_interp = fd.interpolate(w_func, V_coords)
w_at_nodes = w_interp.dat.data_ro

coords_sf = coords_deformed - w_at_nodes  # X = x - w(x)

mesh_sf = fd.Mesh(mesh_obs.coordinates.copy(deepcopy=True))
mesh_sf.coordinates.dat.data[:] = coords_sf

V_sf_rec = fd.assemble(fd.Constant(1.0) * fd.dx(domain=mesh_sf))
V_sf_rec_val = float(V_sf_rec)

print("\nVolúmenes:")
print(f"  V_obs         = {V_obs_val:.4f} mm²")
print(f"  V_sf recup.   = {V_sf_rec_val:.4f} mm²")
print(f"  V_sf verdadero= {V_true_sf_val:.4f} mm²")
print(f"  V_obs/V_sf recup. = {V_obs_val/V_sf_rec_val:.3f}")
print(f"  V_obs/V_sf true   = {V_obs_val/V_true_sf_val:.3f}")
if V_sf_rec_val < V_obs_val:
    print("  ✓ V_sf < V_obs (consistente con configuración libre de tensiones)")
else:
    print("  ⚠ V_sf >= V_obs (revisar cargas/regularización)")

# Error geométrico vs configuración verdadera
error_sf = coords_sf - coords_initial_true
error_sf_mag = np.linalg.norm(error_sf, axis=1)

x_max = coords_deformed[:, 0].max()
y_max = coords_deformed[:, 1].max()
L_char = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)

print("\nError de recuperación (coords):")
print(f"  Error máximo   = {error_sf_mag.max():.6f} mm ({error_sf_mag.max()/L_char*100:.2f} %)")
print(f"  Error promedio = {error_sf_mag.mean():.6f} mm")

w_mag = np.linalg.norm(w_at_nodes, axis=1)
print("\nDesplazamientos:")
print(f"  |u_true|_max = {u_true_mag.max():.4f} mm")
print(f"  |w|_max      = {w_mag.max():.4f} mm")
print(f"  |w|_mean     = {w_mag.mean():.4f} mm")

thetaP_val = thetaP_func.dat.data_ro[0]
thetaM_val = thetaM_func.dat.data_ro[0]
lambda_vol_val = lambda_vol_func.dat.data_ro[0]
print("\nParámetros globales:")
print(f"  θ_P    = {thetaP_val:.4f}")
print(f"  θ_M    = {thetaM_val:.4f}")
print(f"  λ_vol  = {lambda_vol_val:.4e}")

# ============================================================
# 13. GUARDAR RESULTADOS
# ============================================================

output_dir = "phase2_inverse_results"
os.makedirs(output_dir, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO RESULTADOS")
print("="*60)

P_eff = fd.Function(V_P, name="P_eff")
M_eff = fd.Function(V_M, name="M_eff")
P_eff.assign(thetaP_val * P_fwd)
M_eff.assign(thetaM_val * M_fwd)

obs_file = VTKFile(os.path.join(output_dir, "mesh_observed.pvd"))
sf_file = VTKFile(os.path.join(output_dir, "mesh_stressfree.pvd"))

V_obs_out = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
recovery_vec = fd.Function(V_obs_out, name="Recovery_Vector")
recovery_vec.dat.data[:] = -w_at_nodes   # vector desde obs a SF = -w
obs_file.write(recovery_vec, P_eff, M_eff)

V_sf_out = fd.VectorFunctionSpace(mesh_sf, "CG", 1)
zero_sf = fd.Function(V_sf_out, name="zero")
sf_file.write(zero_sf)

np.savez(os.path.join(output_dir, "inverse_data.npz"),
         coords_observed=coords_deformed,
         coords_stressfree=coords_sf,
         coords_stressfree_true=coords_initial_true,
         w_data=w_at_nodes,
         thetaP=thetaP_val,
         thetaM=thetaM_val,
         lambda_vol=lambda_vol_val,
         P_fwd=P_alv_data,
         M_fwd=M_surf_data,
         P_eff=P_eff.dat.data_ro[:],
         M_eff=M_eff.dat.data_ro[:],
         error_sf=error_sf_mag,
         V_obs=V_obs_val,
         V_sf_rec=V_sf_rec_val,
         V_sf_true=V_true_sf_val)

print("\n" + "="*60)
print("RESUMEN FINAL FASE 2 INVERSA (VOL OBJETIVO)")
print("="*60)
print(f"\nVolúmenes:")
print(f"  V_obs       = {V_obs_val:.4f} mm²")
print(f"  V_sf recup. = {V_sf_rec_val:.4f} mm²")
print(f"  V_sf true   = {V_true_sf_val:.4f} mm²")

print(f"\nDesplazamientos:")
print(f"  |u_true|_max = {u_true_mag.max():.4f} mm")
print(f"  |w|_max      = {w_mag.max():.4f} mm")
print(f"  |w|_mean     = {w_mag.mean():.4f} mm")

print(f"\nParámetros:")
print(f"  θ_P    = {thetaP_val:.4f}")
print(f"  θ_M    = {thetaM_val:.4f}")
print(f"  λ_vol  = {lambda_vol_val:.4e}")

print(f"\nErrores geométricos:")
print(f"  Error máx    = {error_sf_mag.max():.4f} mm ({error_sf_mag.max()/L_char*100:.2f} %)")
print(f"  Error medio  = {error_sf_mag.mean():.4f} mm")

print(f"\nArchivos en: {output_dir}/")
print("  - mesh_observed.pvd     (Recovery_Vector, P_eff, M_eff)")
print("  - mesh_stressfree.pvd   (configuración SF recuperada)")
print("  - inverse_data.npz      (datos numéricos)")
print("\nParaView:")
print("  1. Abrir mesh_observed.pvd")
print("  2. Warp By Vector → Recovery_Vector (scale ≈ 1)")
print("  3. Debería contraer hacia la configuración libre de tensiones")
print("\n" + "="*60)
print("FASE 2 INVERSA CON VOLUMEN OBJETIVO COMPLETADA")
print("="*60)


FASE 2: PROBLEMA INVERSO CON θ_P, θ_M + VOLUMEN OBJETIVO

Datos cargados:
  N nodos: 441
  P_alv_final = 0.0000 kPa
  |u_true|_max  = 1.1514 mm
  |u_true|_mean = 0.6982 mm

Volúmenes:
  V_obs (deformado)  = 116.6476 mm²
  V_sf (verdadero)   = 100.0000 mm²
  V_obs/V_sf (esperado) = 1.166

Campos base (Fase 1):
  ||P_fwd||_max = 4.3555e-01 kPa
  ||M_fwd||_max = 7.0154e-08 g/mm²

Gravedad:
  ρ = 350.0 kg/m³
  g = 9.81 m/s²
  Dirección: [ 0. -1.]

RESOLUCIÓN PROBLEMA INVERSO (CONTINUACIÓN EN α)

Paso 1/10: α = 0.10
  0 SNES Function norm 1.664801574699e+01
  1 SNES Function norm 7.706966101611e-01
  2 SNES Function norm 1.894293983003e-02
  3 SNES Function norm 3.822841472111e-04
  4 SNES Function norm 1.757395988826e-07
  ✓ |w|_max = 1.2336e+00 mm, |w|_mean = 6.0074e-01 mm
    θ_P = 1.0000, θ_M = 1.0000, λ_vol = 2.9710e-01

Paso 2/10: α = 0.20
  0 SNES Function norm 1.132869473430e-01
  1 SNES Function norm 1.268160342975e-03
  2 SNES Function norm 5.466626053716e-07
  ✓ |w|_max = 1.2209e

/home/ebasu/venv-firedrake/lib/python3.12/site-packages/firedrake/interpolation.py:391: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.


In [ ]:
"""
FASE 2 (3D): Diagnóstico de malla observada
===========================================

- Carga mesh.h5 (coordenadas + conectividad)
- Construye la malla de Firedrake (mesh_obs)
- Reporta:
    * Nº de celdas y nodos
    * Extensión del dominio en cada eje
    * Volumen total (mm^3 y L)
    * Centro de masa geométrico
    * Estadísticas de volúmenes de celda
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import firedrake as fd
from firedrake.output import VTKFile
from petsc4py import PETSc
import h5py

fd.parameters["form_compiler"]["quadrature_degree"] = 4

# ============================================================
# 1. CARGAR MALLA DESDE mesh.h5
# ============================================================

MESH_FILE = "mesh.h5"

print("="*60)
print("FASE 2: DIAGNÓSTICO DE MALLA 3D")
print("="*60)
print(f"Archivo de malla: {MESH_FILE}")

with h5py.File(MESH_FILE, 'r') as f:
    coords_data = f["mesh"]["coordinates"][:].astype(np.float64)
    cells_data = f["mesh"]["topology"][:].astype(np.int32)

# Construir DMPlex y malla de Firedrake
plex = PETSc.DMPlex().createFromCellList(
    dim=3,              # malla 3D (tetraedros)
    cells=cells_data,
    coords=coords_data,
    comm=PETSc.COMM_WORLD,
)
mesh_obs = fd.Mesh(plex)

coords_obs = mesh_obs.coordinates.dat.data_ro.copy()

# ============================================================
# 2. INFO BÁSICA: N CELDAS, NODOS, EXTENSIÓN
# ============================================================

n_cells = mesh_obs.num_cells()
n_verts = mesh_obs.num_vertices()
dim = mesh_obs.geometric_dimension()

print(f"\nMalla observada (Ω_obs):")
print(f"  Dimensión geométrica = {dim}")
print(f"  Nº de celdas         = {n_cells}")
print(f"  Nº de vértices       = {n_verts}")

# Extensión en cada eje
xmin, ymin, zmin = coords_obs.min(axis=0)
xmax, ymax, zmax = coords_obs.max(axis=0)

Lx = xmax - xmin
Ly = ymax - ymin
Lz = zmax - zmin

print("\nRangos espaciales [mm]:")
print(f"  X: [{xmin:.2f}, {xmax:.2f}]   Lx = {Lx:.2f} mm")
print(f"  Y: [{ymin:.2f}, {ymax:.2f}]   Ly = {Ly:.2f} mm")
print(f"  Z: [{zmin:.2f}, {zmax:.2f}]   Lz = {Lz:.2f} mm")

# Elegir eje vertical “natural” (el de mayor extensión)
L_vec = np.array([Lx, Ly, Lz])
axis_names = ["X", "Y", "Z"]
vert_axis = int(np.argmax(L_vec))
print(f"\nEje de mayor extensión ≈ eje vertical candidato: {axis_names[vert_axis]}")

# ============================================================
# 3. VOLUMEN TOTAL Y CENTRO DE MASA
# ============================================================

X = fd.SpatialCoordinate(mesh_obs)

V_tot = fd.assemble(1.0_



Extensión del dominio [mm]:
  Lx = 130.54, Ly = 171.93, Lz = 213.44


ValueError: Index out of bounds.

In [37]:
"""
FASE 2 (3D): Diagnóstico de malla observada
===========================================

- Carga mesh.h5 (coordenadas + conectividad)
- Construye la malla de Firedrake (mesh_obs)
- Reporta:
    * Nº de celdas y nodos
    * Extensión del dominio en cada eje
    * Volumen total (mm^3 y L)
    * Centro de masa geométrico
    * Estadísticas de volúmenes de celda
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import firedrake as fd
from firedrake.output import VTKFile
from petsc4py import PETSc
import h5py

fd.parameters["form_compiler"]["quadrature_degree"] = 4

# ============================================================
# 1. CARGAR MALLA DESDE mesh.h5
# ============================================================

MESH_FILE = "mesh.h5"

print("="*60)
print("FASE 2: DIAGNÓSTICO DE MALLA 3D")
print("="*60)
print(f"Archivo de malla: {MESH_FILE}")

with h5py.File(MESH_FILE, 'r') as f:
    coords_data = f["mesh"]["coordinates"][:].astype(np.float64)
    cells_data = f["mesh"]["topology"][:].astype(np.int32)

# Construir DMPlex y malla de Firedrake
plex = PETSc.DMPlex().createFromCellList(
    dim=3,              # malla 3D (tetraedros)
    cells=cells_data,
    coords=coords_data,
    comm=PETSc.COMM_WORLD,
)
mesh_obs = fd.Mesh(plex)

coords_obs = mesh_obs.coordinates.dat.data_ro.copy()

# ============================================================
# 2. INFO BÁSICA: N CELDAS, NODOS, EXTENSIÓN
# ============================================================

n_cells = mesh_obs.num_cells()
n_verts = mesh_obs.num_vertices()
dim = mesh_obs.geometric_dimension()

print(f"\nMalla observada (Ω_obs):")
print(f"  Dimensión geométrica = {dim}")
print(f"  Nº de celdas         = {n_cells}")
print(f"  Nº de vértices       = {n_verts}")

# Extensión en cada eje
xmin, ymin, zmin = coords_obs.min(axis=0)
xmax, ymax, zmax = coords_obs.max(axis=0)

Lx = xmax - xmin
Ly = ymax - ymin
Lz = zmax - zmin

print("\nRangos espaciales [mm]:")
print(f"  X: [{xmin:.2f}, {xmax:.2f}]   Lx = {Lx:.2f} mm")
print(f"  Y: [{ymin:.2f}, {ymax:.2f}]   Ly = {Ly:.2f} mm")
print(f"  Z: [{zmin:.2f}, {zmax:.2f}]   Lz = {Lz:.2f} mm")

# Elegir eje vertical “natural” (el de mayor extensión)
L_vec = np.array([Lx, Ly, Lz])
axis_names = ["X", "Y", "Z"]
vert_axis = int(np.argmax(L_vec))
print(f"\nEje de mayor extensión ≈ eje vertical candidato: {axis_names[vert_axis]}")

# ============================================================
# 3. VOLUMEN TOTAL Y CENTRO DE MASA
# ============================================================

X = fd.SpatialCoordinate(mesh_obs)

V_tot = fd.assemble(1.0 * fd.dx(domain=mesh_obs))

# Centroide / centro de masa geométrico (densidad uniforme)
x_cm = [fd.assemble(X[i] * fd.dx(domain=mesh_obs)) / V_tot for i in range(dim)]

print("\nVolumen total:")
print(f"  V_tot = {V_tot:.3e} mm^3  (~ {V_tot/1e6:.4f} L)")

print("\nCentro de masa geométrico [mm]:")
print(f"  x_cm = ({x_cm[0]:.2f}, {x_cm[1]:.2f}, {x_cm[2]:.2f})")

# ============================================================
# 4. ESTADÍSTICAS DE VOLUMEN DE CELDA
# ============================================================

Q_vol = fd.FunctionSpace(mesh_obs, "DG", 0)
cell_vol = fd.Function(Q_vol, name="CellVolume")
cell_vol.interpolate(fd.CellVolume(mesh_obs))

vol_array = cell_vol.dat.data_ro
v_min = vol_array.min()
v_max = vol_array.max()
v_mean = vol_array.mean()

print("\nVolúmenes de celda (tetraedros):")
print(f"  v_min   = {v_min:.3e} mm^3")
print(f"  v_max   = {v_max:.3e} mm^3")
print(f"  v_mean  = {v_mean:.3e} mm^3")
print(f"  Nº celdas= {vol_array.size}")

# ============================================================
# 5. OPCIONAL: EXPORTAR MALLA A VTK PARA VERIFICACIÓN
# ============================================================

outdir = "phase2_mesh_diagnostics"
os.makedirs(outdir, exist_ok=True)

mesh_file = VTKFile(os.path.join(outdir, "mesh_observed.pvd"))
V_vec = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
zero_vec = fd.Function(V_vec, name="zero")
mesh_file.write(zero_vec, cell_vol)

print("\nArchivos generados:")
print(f"  - {outdir}/mesh_observed.pvd  (geometría + CellVolume)")
print("\nEn ParaView:")
print("  1) Abrir mesh_observed.pvd")
print("  2) Colorear por 'CellVolume' para ver la calidad de la malla")
print("  3) Verificar orientación y escala espacial (X,Y,Z en mm)")
print("\nDiagnóstico de malla completado.")


FASE 2: DIAGNÓSTICO DE MALLA 3D
Archivo de malla: mesh.h5

Malla observada (Ω_obs):
  Dimensión geométrica = 3
  Nº de celdas         = 28458
  Nº de vértices       = 5796

Rangos espaciales [mm]:
  X: [-65.27, 65.27]   Lx = 130.54 mm
  Y: [-85.97, 85.97]   Ly = 171.93 mm
  Z: [-106.72, 106.72]   Lz = 213.44 mm

Eje de mayor extensión ≈ eje vertical candidato: Z

Volumen total:
  V_tot = 1.573e+06 mm^3  (~ 1.5731 L)

Centro de masa geométrico [mm]:
  x_cm = (-4.60, 5.50, -5.04)

Volúmenes de celda (tetraedros):
  v_min   = 1.587e+01 mm^3
  v_max   = 1.051e+02 mm^3
  v_mean  = 5.528e+01 mm^3
  Nº celdas= 28458

Archivos generados:
  - phase2_mesh_diagnostics/mesh_observed.pvd  (geometría + CellVolume)

En ParaView:
  1) Abrir mesh_observed.pvd
  2) Colorear por 'CellVolume' para ver la calidad de la malla
  3) Verificar orientación y escala espacial (X,Y,Z en mm)

Diagnóstico de malla completado.


In [ ]:
"""
FASE 2 (3D): Inverse Motion en pulmón con surfactante "base"
============================================================

- Carga la malla 3D del pulmón desde mesh.h5 (Ω_obs)
- Construye campos base:
    * M_surf_fwd(x): masa de surfactante por volumen, con gradiente en z
    * P_alv_fwd(x): presión alveolar uniforme = P_CT
- Resuelve problema inverso:
    Unknowns: w(x), θ_P, θ_M
    Constitutivo: Fung + P_gamma(M_surf) - P_alv
    Gravedad: en eje Z
    Regularización: grad(w)
    Anclaje suave: banda basal (z ~ z_min)

Salida:
    - phase2_inverse_lung/mesh_observed.pvd
        * Recovery_Vector (vector hacia config SF)
        * P_eff, M_eff
    - phase2_inverse_lung/mesh_stressfree.pvd
        * geometría SF
    - phase2_inverse_lung/inverse_lung_data.npz
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import firedrake as fd
from firedrake.output import VTKFile
from petsc4py import PETSc
import h5py

fd.parameters["form_compiler"]["quadrature_degree"] = 4

# ============================================================
# 1. CARGAR MALLA DEL PULMÓN (Ω_obs)
# ============================================================

MESH_FILE = "mesh.h5"
print("="*60)
print("FASE 2 (3D): INVERSE MOTION EN PULMÓN CON SURFACTANTE BASE")
print("="*60)
print(f"Archivo de malla: {MESH_FILE}")

with h5py.File(MESH_FILE, 'r') as f:
    coords_data = f['mesh']['coordinates'][:].astype(np.float64)
    cells_data = f['mesh']['topology'][:].astype(np.int32)

plex = PETSc.DMPlex().createFromCellList(3, cells_data, coords_data, comm=PETSc.COMM_WORLD)
mesh_obs = fd.Mesh(plex)

coords_obs = mesh_obs.coordinates.dat.data_ro.copy()

dim = mesh_obs.geometric_dimension()
print(f"\nMalla observada (Ω_obs):")
print(f"  Dimensión geométrica = {dim}")
print(f"  Nº de celdas         = {mesh_obs.num_cells()}")
print(f"  Nº de vértices       = {mesh_obs.num_vertices()}")

x_min = coords_obs[:, 0].min()
x_max = coords_obs[:, 0].max()
y_min = coords_obs[:, 1].min()
y_max = coords_obs[:, 1].max()
z_min = coords_obs[:, 2].min()
z_max = coords_obs[:, 2].max()

Lx = x_max - x_min
Ly = y_max - y_min
Lz = z_max - z_min

print("\nRangos espaciales [mm]:")
print(f"  X: [{x_min:.2f}, {x_max:.2f}]   Lx = {Lx:.2f} mm")
print(f"  Y: [{y_min:.2f}, {y_max:.2f}]   Ly = {Ly:.2f} mm")
print(f"  Z: [{z_min:.2f}, {z_max:.2f}]   Lz = {Lz:.2f} mm")
print("\nTomamos Z como eje vertical (gravedad).")

V_tot = fd.assemble(fd.Constant(1.0) * fd.dx(domain=mesh_obs))
print(f"\nVolumen total:")
print(f"  V_tot = {V_tot:.3e} mm^3 (~ {V_tot/1e6:.4f} L)")

# ============================================================
# 2. PARÁMETROS DEL MODELO (MISMO ESTILO QUE ALVEOLO)
# ============================================================

# Geometría alveolar efectiva
Phi_R = 0.74
R_RI = 0.11

# Material Fung
c_fung = 2.5
a_fung = 0.433
b_fung = -0.61

# Surfactante
gamma_0 = 70.0e-6
gamma_inf = 22.2e-6
gamma_min = 0.0
Gamma_inf = 3.0e-9
m1 = 47.8e-6
m2 = 140.0e-6
Gamma_max = Gamma_inf * (1.0 + (gamma_inf - gamma_min) / m2)

# Densidad de tejido y gravedad (en mm, kPa)
rho_tissue = 350e-9   # kg/mm^3
g_val = 9810.0        # mm/s^2
gravity_direction = np.array([0.0, 0.0, -1.0])  # Z hacia abajo

print("\nParámetros materiales (resumen):")
print(f"  Phi_R = {Phi_R}, R_RI = {R_RI} mm")
print(f"  Fung: c = {c_fung}, a = {a_fung}, b = {b_fung}")
print(f"  Surfactante: Gamma_inf = {Gamma_inf:.3e}, Gamma_max = {Gamma_max:.3e}")
print(f"  ρ_tissue = {rho_tissue*1e9:.1f} kg/m^3, g = {g_val/1000:.2f} m/s^2")

# ============================================================
# 3. CAMPOS BASE M_surf_fwd(x) Y P_alv_fwd(x)
# ============================================================

# --- Espacios ---
V_M = fd.FunctionSpace(mesh_obs, "DG", 0)   # M_surf (masa/volumen)
V_P = fd.FunctionSpace(mesh_obs, "CG", 1)   # P_alv

M_fwd = fd.Function(V_M, name="M_forward")
P_fwd = fd.Function(V_P, name="P_forward")

x = fd.SpatialCoordinate(mesh_obs)
z = x[2]

# --- M_surf_fwd con gradiente en z ---

# A_density_ref = (3/R_RI) * Phi_R^(1/3) * Phi_R^(2/3) = (3/R_RI) * Phi_R
A_density_ref = (3.0 / R_RI) * Phi_R
A_density_ref_c = fd.Constant(A_density_ref)
Gamma_max_c = fd.Constant(Gamma_max)

# mapeo z -> [0,1]
z_min_val = float(z_min)
z_max_val = float(z_max)
xi = (z - z_min_val) / (z_max_val - z_min_val)

# variación vertical (20%)
alpha_Gamma = 0.2
alpha_Gamma_c = fd.Constant(alpha_Gamma)

Gamma_base_expr = Gamma_max_c * (1.0 - alpha_Gamma_c * (xi - 0.5))
M_base_expr = Gamma_base_expr * A_density_ref_c

M_fwd.assign(fd.project(M_base_expr, V_M))

# --- P_alv_fwd(x) uniforme = P_CT ---

# Ajusta P_CT_val según tu protocolo de tomografía (en kPa),
# como primer tiro puedes poner 0.5 o 1.0 kPa.
P_CT_val = 0.5
P_CT = fd.Constant(P_CT_val)
P_fwd.assign(P_CT)

print("\nCampos base construidos:")
print(f"  M_fwd: min = {M_fwd.dat.data_ro.min():.3e}, max = {M_fwd.dat.data_ro.max():.3e} [masa/vol]")
print(f"  P_fwd: valor uniforme = {P_CT_val:.3f} kPa")

# ============================================================
# 4. ESPACIO MIXTO PARA (w, θ_P, θ_M)
# ============================================================

V_w = fd.VectorFunctionSpace(mesh_obs, "CG", 2)
R_space = fd.FunctionSpace(mesh_obs, "R", 0)

W = V_w * R_space * R_space
state = fd.Function(W, name="Inverse_State")
w, theta_P, theta_M = fd.split(state)
v_w, q_P, q_M = fd.TestFunctions(W)

w_func, thetaP_func, thetaM_func = state.subfunctions

# Constantes UFL
rho_R = fd.Constant(rho_tissue)
g_vec = fd.Constant(tuple(g_val * gravity_direction))

Phi_R_c = fd.Constant(Phi_R)
R_RI_c = fd.Constant(R_RI)
c_fung_c = fd.Constant(c_fung)
a_fung_c = fd.Constant(a_fung)
b_fung_c = fd.Constant(b_fung)
Gamma_inf_c = fd.Constant(Gamma_inf)
gamma_0_c = fd.Constant(gamma_0)
gamma_inf_c = fd.Constant(gamma_inf)
gamma_min_c = fd.Constant(gamma_min)
m1_c = fd.Constant(m1)
m2_c = fd.Constant(m2)

# Regularizaciones
alpha_reg_w = fd.Constant(1e-2)   # suavidad de w
lambda_P_reg = fd.Constant(1e-2)  # Tikhonov para θ_P
lambda_M_reg = fd.Constant(1e-2)  # Tikhonov para θ_M

# Parámetro de continuación en las cargas
alpha_load = fd.Constant(0.0)

# ============================================================
# 5. CINEMÁTICA INVERSE MOTION + CONSTITUTIVO
# ============================================================

I = fd.Identity(dim)
grad_w = fd.grad(w)
f = I - grad_w
det_f = fd.det(f)

F = fd.inv(f)
J = fd.det(F)
C = F.T * F
invC = fd.inv(C)

E = fd.variable(0.5 * (C - I))
J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))

Psi_el = c_fung_c * (fd.exp(a_fung_c * J1**2 + b_fung_c * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

Phi = fd.max_value(J - 1.0 + Phi_R_c, 1e-4)
A_density = (3.0 / R_RI_c) * Phi_R_c**(1.0/3.0) * Phi**(2.0/3.0)

P_alv_eff = theta_P * P_fwd
M_surf_eff = theta_M * M_fwd

Gamma = M_surf_eff / fd.max_value(A_density, 1e-12)

ratio = Gamma / Gamma_inf_c
gamma_lang = gamma_0_c - m1_c * ratio
gamma_insol = gamma_inf_c - m2_c * (ratio - 1.0)
gamma_surf = fd.conditional(fd.lt(Gamma, Gamma_inf_c), gamma_lang, gamma_insol)
gamma_surf = fd.max_value(gamma_surf, gamma_min_c)

P_gamma = (2.0 * gamma_surf / R_RI_c) * ((Phi_R_c / Phi)**(1.0/3.0))

S_total = S_el + alpha_load * (P_gamma - P_alv_eff) * J * invC
tau = F * S_total * F.T

body_force = alpha_load * rho_R * g_vec

# ============================================================
# 6. FORMULACIÓN VARIACIONAL EN Ω_obs
# ============================================================

Residual = fd.inner(tau, fd.grad(v_w)) * det_f * fd.dx \
         - fd.inner(body_force, v_w) * det_f * fd.dx

# --- Anclaje suave en banda basal (z cerca de z_min) ---
tol_z = 0.05 * (z_max - z_min)
indicator_base = fd.conditional(fd.lt(z, z_min_val + tol_z), 1.0, 0.0)
beta_anchor = fd.Constant(1e-3)  # anclaje suave

Residual += beta_anchor * indicator_base * fd.inner(w, v_w) * fd.dx

# --- Regularización de w (tipo H1) ---
Residual += alpha_reg_w * fd.inner(fd.grad(w), fd.grad(v_w)) * fd.dx

# --- Ecuaciones suaves para θ_P y θ_M (Tikhonov hacia 1) ---
Residual += lambda_P_reg * (theta_P - 1.0) * q_P * fd.dx
Residual += lambda_M_reg * (theta_M - 1.0) * q_M * fd.dx

# ============================================================
# 7. SOLVER NO LINEAL (FIELDSPLIT CON ESPACIO R)
# ============================================================

problem = fd.NonlinearVariationalProblem(Residual, state)

solver_params = {
    # --- ANDERSON / NGMRES ---
    "snes_type": "ngmres",                # Nonlinear GMRES (Anderson mixing)
    "snes_ngmres_m": 20,                  # largo de history (10–30 típico)
    "snes_ngmres_monitor": None,          # para ver el residuo de ngmres
    "snes_max_it": 80,                    # más iteraciones si hace falta
    "snes_atol": 1e-6,
    "snes_rtol": 1e-5,

    # (opcional pero útil: un poco de line search)
    "snes_linesearch_type": "bt",         # backtracking
    "snes_linesearch_max_it": 3,

    # --- LINEAR SOLVER / PREC como antes ---
    "mat_type": "nest",
    "ksp_type": "fgmres",
    "ksp_max_it": 20,

    "pc_type": "fieldsplit",
    "pc_fieldsplit_type": "multiplicative",

    # Bloque 0: w
    "fieldsplit_0_ksp_type": "preonly",
    "fieldsplit_0_pc_type": "lu",
    "fieldsplit_0_pc_factor_mat_solver_type": "mumps",

    # Bloque 1: θ_P
    "fieldsplit_1_ksp_type": "preonly",
    "fieldsplit_1_pc_type": "jacobi",

    # Bloque 2: θ_M
    "fieldsplit_2_ksp_type": "preonly",
    "fieldsplit_2_pc_type": "jacobi",
}


solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# Inicialización
w_func.assign(fd.Constant((0.0, 0.0, 0.0)))
thetaP_func.assign(1.0)
thetaM_func.assign(1.0)

# ============================================================
# 8. CONTINUACIÓN EN α (CARGAS) ADAPTATIVA
# ============================================================

print("\n" + "="*60)
print("RESOLUCIÓN PROBLEMA INVERSO (CONTINUACIÓN EN α)")
print("="*60)

alpha_val = 0.0          # α actual (Python float)
dalpha = 0.1             # paso inicial
alpha_min = 1e-3         # paso mínimo permitido
alpha_max = 0.3          # paso máximo permitido
step = 0

while alpha_val < 1.0 - 1e-12:
    alpha_trial = min(1.0, alpha_val + dalpha)
    alpha_load.assign(alpha_trial)
    step += 1

    print(f"\nPaso {step}: α = {alpha_trial:.3f} (Δα = {dalpha:.3f})")
    try:
        solver.solve()
        its = solver.snes.getIterationNumber()

        w_data = w_func.dat.data_ro
        w_mag = np.linalg.norm(w_data, axis=1)
        thetaP_val = float(thetaP_func.dat.data_ro[0])
        thetaM_val = float(thetaM_func.dat.data_ro[0])

        print(f"  ✓ Convergió con {its} iteraciones Newton")
        print(f"    |w|_max = {w_mag.max():.4e} mm, |w|_mean = {w_mag.mean():.4e} mm")
        print(f"    θ_P = {thetaP_val:.4f}, θ_M = {thetaM_val:.4f}")

        # Acepta el paso
        alpha_val = alpha_trial

        # Adaptación de Δα según dificultad
        if its <= 4 and dalpha < alpha_max:
            dalpha *= 1.5   # problema fácil → agrandar paso
        elif its > 8:
            dalpha *= 0.7   # problema duro pero converge → achicar

    except fd.ConvergenceError:
        print("  ✗ No convergió, reduciendo Δα y reintentando")
        alpha_load.assign(alpha_val)
        dalpha *= 0.5
        if dalpha < alpha_min:
            raise RuntimeError("No puedo avanzar en α: Δα demasiado pequeño.")

# ============================================================
# 9. CONFIGURACIÓN STRESS-FREE RECUPERADA
# ============================================================

print("\n" + "="*60)
print("CONFIGURACIÓN STRESS-FREE RECUPERADA (PULMÓN)")
print("="*60)

V_coords = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
w_interp = fd.interpolate(w_func, V_coords)
w_at_nodes = w_interp.dat.data_ro.copy()

coords_sf = coords_obs - w_at_nodes

mesh_sf = fd.Mesh(mesh_obs.coordinates.copy(deepcopy=True))
mesh_sf.coordinates.dat.data[:] = coords_sf

V_obs_val = V_tot
V_sf_val = fd.assemble(fd.Constant(1.0) * fd.dx(domain=mesh_sf))

print("\nVolúmenes:")
print(f"  V_obs (CT)     = {V_obs_val:.3e} mm^3 (~ {V_obs_val/1e6:.4f} L)")
print(f"  V_sf recup.    = {V_sf_val:.3e} mm^3 (~ {V_sf_val/1e6:.4f} L)")
print(f"  Ratio V_obs/V_sf = {V_obs_val/V_sf_val:.3f}")

w_mag = np.linalg.norm(w_at_nodes, axis=1)
print("\nDesplazamientos inversos:")
print(f"  |w|_max  = {w_mag.max():.4f} mm")
print(f"  |w|_mean = {w_mag.mean():.4f} mm")

thetaP_val = float(thetaP_func.dat.data_ro[0])
thetaM_val = float(thetaM_func.dat.data_ro[0])
print("\nParámetros globales estimados:")
print(f"  θ_P = {thetaP_val:.4f}")
print(f"  θ_M = {thetaM_val:.4f}")

# ============================================================
# 10. SALIDAS VTK + NPZ
# ============================================================

output_dir = "phase2_inverse_lung"
os.makedirs(output_dir, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO RESULTADOS")
print("="*60)

# Campos efectivos
P_eff = fd.Function(V_P, name="P_eff")
M_eff = fd.Function(V_M, name="M_eff")
P_eff.assign(thetaP_val * P_fwd)
M_eff.assign(thetaM_val * M_fwd)

# Observed mesh + recovery vector
obs_file = VTKFile(os.path.join(output_dir, "mesh_observed.pvd"))
V_obs_vec = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
recovery_vec = fd.Function(V_obs_vec, name="Recovery_Vector")
recovery_vec.dat.data[:] = -w_at_nodes   # vector desde Ω_obs hacia Ω_sf

obs_file.write(recovery_vec, P_eff, M_eff)

# Stress-free mesh
sf_file = VTKFile(os.path.join(output_dir, "mesh_stressfree.pvd"))
V_sf_vec = fd.VectorFunctionSpace(mesh_sf, "CG", 1)
zero_sf = fd.Function(V_sf_vec, name="zero")
sf_file.write(zero_sf)

# Guardar datos numéricos
np.savez(os.path.join(output_dir, "inverse_lung_data.npz"),
         coords_observed=coords_obs,
         coords_stressfree=coords_sf,
         w_data=w_at_nodes,
         thetaP=thetaP_val,
         thetaM=thetaM_val,
         P_fwd=P_fwd.dat.data_ro[:],
         M_fwd=M_fwd.dat.data_ro[:],
         P_eff=P_eff.dat.data_ro[:],
         M_eff=M_eff.dat.data_ro[:],
         V_obs=V_obs_val,
         V_sf=V_sf_val)

print("\n" + "="*60)
print("RESUMEN FINAL FASE 2 (PULMÓN)")
print("="*60)
print(f"\nVolúmenes:")
print(f"  V_obs  = {V_obs_val:.3e} mm^3 (~ {V_obs_val/1e6:.4f} L)")
print(f"  V_sf   = {V_sf_val:.3e} mm^3 (~ {V_sf_val/1e6:.4f} L)")
print(f"  Ratio  = V_obs/V_sf = {V_obs_val/V_sf_val:.3f}")

print(f"\nDesplazamientos w:")
print(f"  |w|_max  = {w_mag.max():.4f} mm")
print(f"  |w|_mean = {w_mag.mean():.4f} mm")

print(f"\nParámetros:")
print(f"  θ_P = {thetaP_val:.4f}")
print(f"  θ_M = {thetaM_val:.4f}")

print(f"\nArchivos en: {output_dir}/")
print("  - mesh_observed.pvd     (Recovery_Vector, P_eff, M_eff)")
print("  - mesh_stressfree.pvd   (configuración SF recuperada)")
print("  - inverse_lung_data.npz (datos numéricos)")
print("\nEn ParaView:")
print("  1. Abrir mesh_observed.pvd")
print("  2. Warp By Vector -> Recovery_Vector (escala ≈ 1)")
print("  3. Ver contracción hacia la configuración libre de tensiones")
print("="*60)
print("FASE 2 (PULMÓN) COMPLETADA")
print("="*60)


FASE 2 (3D): INVERSE MOTION EN PULMÓN CON SURFACTANTE BASE
Archivo de malla: mesh.h5

Malla observada (Ω_obs):
  Dimensión geométrica = 3
  Nº de celdas         = 28458
  Nº de vértices       = 5796

Rangos espaciales [mm]:
  X: [-65.27, 65.27]   Lx = 130.54 mm
  Y: [-85.97, 85.97]   Ly = 171.93 mm
  Z: [-106.72, 106.72]   Lz = 213.44 mm

Tomamos Z como eje vertical (gravedad).

Volumen total:
  V_tot = 1.573e+06 mm^3 (~ 1.5731 L)

Parámetros materiales (resumen):
  Phi_R = 0.74, R_RI = 0.11 mm
  Fung: c = 2.5, a = 0.433, b = -0.61
  Surfactante: Gamma_inf = 3.000e-09, Gamma_max = 3.476e-09
  ρ_tissue = 350.0 kg/m^3, g = 9.81 m/s^2

Campos base construidos:
  M_fwd: min = 6.324e-08, max = 7.699e-08 [masa/vol]
  P_fwd: valor uniforme = 0.500 kPa

RESOLUCIÓN PROBLEMA INVERSO (CONTINUACIÓN EN α)

Paso 1: α = 0.100 (Δα = 0.100)
picked X_A, obj(X_A) = 3.943972e+01, ||F_A||_2 = 3.943972e+01, obj(X_M) = 4.113625e+01, ||F_M||_2 = 4.113625e+01
picked X_A, obj(X_A) = 3.301440e+01, ||F_A||_2 = 3.